In [ ]:
#!/usr/bin/env python3
#=============================================================================#
# Author    : DaeHee Kim
# Date      : 2025-05-30
# Usage     : RNA-seq 자동화 개선버전
# Example   : 
# Description   : 
#=============================================================================#

"""

1. GSM을 SRR로 변환
2. SRR을 prefetch
3. SRA 파일 fasterq-dump
4. STAR로 align, fastq 파일 삭제, bam 파일 삭제
5. ReadsPerGene 파일 각 GSM 별로 이동
6. Deseq2 분석

"""



import glob, random, os, time, argparse, requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict


file = "test_dataset.xlsx"
core = 32
genome = "hs"

if genome == "mm":
    star_index = "/program/STAR_index/mm10"
elif genome == "hs":
    star_index = "/program/STAR_index/hg38"
elif genome == "sc":
    star_index = "/program/STAR_index/r64"
elif genome == "dm":
    star_index = "/program/STAR_index/BDGP6"
elif genome == "mg":
    star_index = "/program/STAR_index/mg10"
elif genome == "xl":
    star_index = "/program/STAR_index/xl10"
    
if not os.path.isdir("1_gz"):
    os.mkdir("1_gz")
if not os.path.isdir("2_fastq"):
    os.mkdir("2_fastq")
if not os.path.isdir("3_aligned"):
    os.mkdir("3_aligned")
if not os.path.isdir("4_ReadsPerGenes"):
    os.mkdir("4_ReadsPerGenes")
if not os.path.isdir("5_ReadsPerGenes_GSM"):
    os.mkdir("5_ReadsPerGenes_GSM")
if not os.path.isdir("6_deseq2_results"):
    os.mkdir("6_deseq2_results")

with open("log", "w") as f:
    pass 
    
#=============================================================================#

# 출력 및 로그 작성용 함수
def log_writer(text, line=False):
    with open("log", "a") as f:
        if line:
            f.write("#"*80)
            f.write("\n")
        else:
            f.write("\n")
        #print(text)
        f.write(text)
        f.write("\n")
        
#=============================================================================#

# 시작 파일은 GSE \t GSM,GSM \t GSM,GSM 으로 구성
start_time = time.strftime('%Y-%m-%d %H:%M:%S')
print(start_time)

df = pd.read_excel(file,header=0, names = ['GSE', 'Treat','Control'])

grouped_data = defaultdict(list)
for name, group in df.groupby('GSE'):
    grouped_data[name] = group.to_dict(orient='records')

#=============================================================================#

# GSM을 SRR로 변환하는 함수
def gsm_to_srr(gsm):
    treat_url = "https://trace.ncbi.nlm.nih.gov/Traces/sra/?sp=runinfo&acc=" + gsm
    try:
        response = requests.get(treat_url)
    except:
        log_writer(f"Gathering {gsm} info failed", True)

    if response.status_code == 200:
        lines = response.text.strip().split("\n")
    else:
        print("Failed to retrieve data: Status code", response.status_code)
        
    for line in lines[1:]:
        line = line.strip().split(",")
        srr_num = line[0]
        return(srr_num)

#=============================================================================#

# 겹치는 GSE 구분
gse_counts = df['GSE'].value_counts()
gse_index = df.groupby('GSE').cumcount() + 1

def make_gse_unique(row):
    if gse_counts[row['GSE']] == 1:
        return row['GSE']
    else:
        return f"{row['GSE']}_{row['gse_index']}"

df['gse_index'] = gse_index
df['GSE_unique'] = df.apply(make_gse_unique, axis=1)
df = df.drop(columns=['gse_index'])



2025-06-09 12:05:21


In [9]:
#=============================================================================#

# GSM to SRR 변환
gse_gsm_dict = df.groupby('GSE').agg({
    'Treat': lambda x: x.iloc[0].split(','),
    'Control': lambda x: x.iloc[0].split(',')
    
}).to_dict(orient='index')

gsm_to_srr_i = 0
gsm_to_srr_dict = defaultdict(list)
for gse, cond_dict in gse_gsm_dict.items():
    gsms = cond_dict['Treat'] + cond_dict['Control']
    
    for gsm in gsms:
        gsm_to_srr_i += 1
        print(f"Converting {gsm_to_srr_i} GSM to SRR                   ", end="\r")
        srr = gsm_to_srr(gsm)
        if srr:
            gsm_to_srr_dict[gsm].append(srr)
        else:
            log_writer(f"Failed to convert {gsm} to SRR", True)

print(gsm_to_srr_dict)  

#=============================================================================#

defaultdict(<class 'list'>, {'GSM3416106': ['SRR7962899'], 'GSM3416107': ['SRR7962900'], 'GSM3416108': ['SRR7962901'], 'GSM3416109': ['SRR7962902'], 'GSM3416110': ['SRR7962903'], 'GSM3416076': ['SRR7962869'], 'GSM3416077': ['SRR7962870'], 'GSM3416078': ['SRR7962871'], 'GSM3416079': ['SRR7962872'], 'GSM3416080': ['SRR7962873'], 'GSM4321727': ['SRR11116155'], 'GSM4321728': ['SRR11116156'], 'GSM4321729': ['SRR11116157'], 'GSM4321730': ['SRR11116158'], 'GSM4321723': ['SRR11116151'], 'GSM4321724': ['SRR11116152'], 'GSM4321725': ['SRR11116153'], 'GSM4321726': ['SRR11116154'], 'GSM4667244': ['SRR12196459'], 'GSM4667245': ['SRR12196460'], 'GSM4667246': ['SRR12196461'], 'GSM4667247': ['SRR12196462'], 'GSM4667236': ['SRR12196451'], 'GSM4667237': ['SRR12196452'], 'GSM4667238': ['SRR12196453'], 'GSM4667239': ['SRR12196454'], 'GSM1836110': ['SRR2569772'], 'GSM1836111': ['SRR2569773'], 'GSM1836112': ['SRR2569774'], 'GSM1836104': ['SRR2569766'], 'GSM1836105': ['SRR2569767'], 'GSM1836106': ['SRR256976